In [3]:
import pandas as pd

In [4]:
csv_file= "https://drive.google.com/uc?id=1_rK8orgmy-GnfKAkqh5qHk-6s_pDnGxj"
df = pd.read_csv(csv_file)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ID                          30000 non-null  int64  
 1   LIMIT_BAL                   30000 non-null  float64
 2   SEX                         30000 non-null  int64  
 3   EDUCATION                   30000 non-null  int64  
 4   MARRIAGE                    30000 non-null  int64  
 5   AGE                         30000 non-null  int64  
 6   PAY_0                       30000 non-null  int64  
 7   PAY_2                       30000 non-null  int64  
 8   PAY_3                       30000 non-null  int64  
 9   PAY_4                       30000 non-null  int64  
 10  PAY_5                       30000 non-null  int64  
 11  PAY_6                       30000 non-null  int64  
 12  BILL_AMT1                   30000 non-null  float64
 13  BILL_AMT2                   300

In [7]:
df.rename(columns={"PAY_0": "PAY_1"}, inplace=True)

In [8]:
df.rename(columns={"default.payment.next.month": "DEFAULT"}, inplace=True)

#### Find out which type has more defaulters
#### 1 = Male , 2 = Female

In [9]:
#Calculater number of male and female clients out of all defaulters
df[["ID","SEX","DEFAULT"]].groupby(by=["DEFAULT","SEX"]).count()

ID
DEFAULT SEX       
0       1     9015
        2    14349
1       1     2873
        2     3763

In [12]:
#Calculate number of clients by education level out of all defaulters
df[["ID","EDUCATION","DEFAULT"]].groupby(by=["DEFAULT","EDUCATION"]).count()

ID
DEFAULT EDUCATION       
0       0             14
        1           8549
        2          10700
        3           3680
        4            116
        5            262
        6             43
1       1           2036
        2           3330
        3           1237
        4              7
        5             18
        6              8

In [13]:
# Group education levels: 0, 4, 5, 6 -> "Others"
# 1 -> Graduate, 2 -> University, 3 -> High School
education_mapping = {0: "Others", 1: "Graduate", 2: "University", 3: "High School", 4: "Others", 5: "Others", 6: "Others"}
df["EDUCATION_GROUPED"] = df["EDUCATION"].map(education_mapping)

# Now group by DEFAULT and new EDUCATION_GROUPED column
df[["ID","EDUCATION_GROUPED","DEFAULT"]].groupby(by=["DEFAULT","EDUCATION_GROUPED"]).count()

ID
DEFAULT EDUCATION_GROUPED       
0       Graduate            8549
        High School         3680
        Others               435
        University         10700
1       Graduate            2036
        High School         1237
        Others                33
        University          3330

In [14]:
# Calculate defaulter percentages by gender
sex_default = df[["ID","SEX","DEFAULT"]].groupby(by=["DEFAULT","SEX"]).count()

# Get counts for males (SEX=1) and females (SEX=2)
male_total = sex_default.loc[(0, 1), "ID"] + sex_default.loc[(1, 1), "ID"]
female_total = sex_default.loc[(0, 2), "ID"] + sex_default.loc[(1, 2), "ID"]

male_defaulters = sex_default.loc[(1, 1), "ID"]
female_defaulters = sex_default.loc[(1, 2), "ID"]

# Calculate percentages
t_male = (male_defaulters / male_total) * 100
t_female = (female_defaulters / female_total) * 100

print(f"Male defaulter percentage: {t_male:.2f}%")
print(f"Female defaulter percentage: {t_female:.2f}%")

Male defaulter percentage: 24.17%
Female defaulter percentage: 20.78%


In [16]:
# Find types of defaulters by education and marital status
# Filter only defaulters (DEFAULT = 1)
defaulters = df[df["DEFAULT"] == 1]

# Group by EDUCATION and MARRIAGE
defaulters_by_edu_marriage = defaulters[["ID","EDUCATION","MARRIAGE"]].groupby(by=["EDUCATION","MARRIAGE"]).count()
print("Defaulters by Education and Marital Status:")
print(defaulters_by_edu_marriage)

Defaulters by Education and Marital Status:
                      ID
EDUCATION MARRIAGE      
1         1          769
          2         1254
          3           13
2         0            1
          1         1666
          2         1619
          3           44
3         0            4
          1          754
          2          452
          3           27
4         1            3
          2            4
5         1            9
          2            9
6         1            5
          2            3
